<a href="https://colab.research.google.com/github/Soarbh-Srivastava/Extension/blob/main/Modals_ProtoType.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install tensorflow


In [ ]:
from google.colab import drive
from joblib import dump

In [ ]:
#Run to remove drive
from google.colab import drive
drive.flush_and_unmount()


Drive not mounted, so nothing to flush and unmount.


In [ ]:
#Run to import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [39]:
# Navie_Bayes Modal

# Block 1: Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from joblib import dump

# Block 2: Load and Preprocess Data (Updated for TSV)
# You can manually upload the TSV file or provide the path to an existing file
file_path_tsv = "/content/drive/MyDrive/dataset.tsv"  # Replace with the actual path

# Read the TSV file (specify the delimiter as '\t' for tab-separated values)
df = pd.read_csv(file_path_tsv, delimiter='\t')

# Ensure the column names match your actual column names
df.columns = ['page_id', 'text', 'label', 'category']

# Preprocess data
df = df[pd.notnull(df["text"])]
df = df[df["label"].apply(lambda x: str(x).isdigit())]

texts = df['text'].tolist()
labels = df['label'].astype(int).tolist()

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Block 3: Train Naive Bayes Classifier
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_texts)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = BernoulliNB().fit(X_train_tfidf, train_labels)

# Block 4: Evaluate and Save Model
# Predict on the validation set
X_val_counts = count_vect.transform(val_texts)
X_val_tfidf = tfidf_transformer.transform(X_val_counts)
y_pred = clf.predict(X_val_tfidf)

# Evaluate the model
accuracy = metrics.accuracy_score(y_pred, val_labels)
print("Accuracy: ", accuracy)

import os
from joblib import dump

# Define the path to the folder where you want to save the files in the Colab environment
colab_folder_path = '/content/Naive_bayes'

# Create the folder if it doesn't exist
os.makedirs(colab_folder_path, exist_ok=True)

# Save the trained model and vectorizer to the specified folder
dump(clf, os.path.join(colab_folder_path, 'text_classifier.joblib'))
dump(count_vect, os.path.join(colab_folder_path, 'text_vectorizer.joblib'))






Accuracy:  0.9449152542372882


['/content/Naive_bayes/text_vectorizer.joblib']

In [26]:
# Block 5: Test the Model with Sample Texts
sample_texts = [
    "Limited stock available, buy now!",
    "Free trial for 30 days, cancel anytime.",
    "Write a review and get a discount!",
    "Click here to win a prize!",

]

# Preprocess the sample texts
sample_texts = [text.lower() for text in sample_texts]  # Convert to lowercase for consistency
sample_texts_counts = count_vect.transform(sample_texts)
sample_texts_tfidf = tfidf_transformer.transform(sample_texts_counts)

# Predict using the trained model
sample_logits = clf.predict_log_proba(sample_texts_tfidf)  # Log probabilities
sample_probs = torch.softmax(torch.tensor(sample_logits), dim=1).numpy()  # Probabilities
sample_predictions = clf.predict(sample_texts_tfidf)

# Display the results
for text, prediction, prob in zip(sample_texts, sample_predictions, sample_probs[:, 1]):
    # Convert probability to percentage
    trust_score_percentage = (1 - prob) * 100

    # Set a minimum trust score for non-dark patterns
    min_trust_score = 10 if prediction == 0 else 0

    # Ensure that trust score is at least the minimum value
    trust_score_percentage = max(trust_score_percentage, min_trust_score)

    category = "Dark Pattern" if prediction == 1 else "Not Dark Pattern"
    print(f"Text: {text}\nPrediction: {category}\nTrust Score: {trust_score_percentage:.2f}%\n")


Text: limited stock available, buy now!
Prediction: Dark Pattern
Trust Score: 0.02%

Text: free trial for 30 days, cancel anytime.
Prediction: Dark Pattern
Trust Score: 0.04%

Text: write a review and get a discount!
Prediction: Not Dark Pattern
Trust Score: 95.99%

Text: click here to win a prize!
Prediction: Not Dark Pattern
Trust Score: 97.90%



In [27]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load TSV file from Google Drive
file_path_tsv = "/content/drive/MyDrive/dataset.tsv"
df = pd.read_csv(file_path_tsv, delimiter='\t')

# Assuming you have 'text' and 'label' columns
texts = df['text'].tolist()
labels = df['label'].tolist()

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Convert the labels to tensors
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create DataLoader for training and validation sets
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              train_labels)
val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']),
                            torch.tensor(val_encodings['attention_mask']),
                            val_labels)

# Define batch size
batch_size = 8

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Training loop with learning rate adjustment, early stopping, and dropout regularization
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

num_epochs = 10  # Increase the number of epochs as needed
best_accuracy = 0.0
patience = 3
dropout_prob = 0.1  # Adjust as needed

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Training
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    all_preds = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())

    accuracy = accuracy_score(val_labels.numpy(), all_preds)
    print(f'Validation Accuracy: {accuracy:.4f}')

    # Early stopping check
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        # Save the model or any other necessary actions
        model.save_pretrained('distilbert_text_classifier')
        tokenizer.save_pretrained('distilbert_text_classifier')
        print("Model saved with the best accuracy.")

    else:
        patience -= 1
        if patience == 0:
            print("Early stopping. No improvement in validation accuracy.")
            break

    # Learning rate adjustment
    if epoch == 2:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 1e-5  # Adjust as needed


    # Regularization - Apply dropout
    model.distilbert.embeddings.dropout.p = dropout_prob
    for layer in model.distilbert.transformer.layer:
        layer.attention.dropout.p = dropout_prob
        layer.sa_layer_norm.p = dropout_prob

# Check if the classifier layer has a dropout attribute
    if hasattr(model.pre_classifier, 'dropout'):
        model.pre_classifier.dropout.p = dropout_prob
        model.classifier.dropout.p = dropout_prob


# Save the final model
model.save_pretrained('distilbert_text_classifier')
tokenizer.save_pretrained('distilbert_text_classifier')


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
Validation Accuracy: 0.9703
Model saved with the best accuracy.
Epoch 2/10
Validation Accuracy: 0.9597
Epoch 3/10
Validation Accuracy: 0.9619
Epoch 4/10
Validation Accuracy: 0.9703
Early stopping. No improvement in validation accuracy.


('distilbert_text_classifier/tokenizer_config.json',
 'distilbert_text_classifier/special_tokens_map.json',
 'distilbert_text_classifier/vocab.txt',
 'distilbert_text_classifier/added_tokens.json')

In [34]:
# Test the Model with Sample Texts
sample_texts = [
    "Limited stock available, buy now!",
    "Free trial for 30 days, cancel anytime.",
    "Write a review and get a discount!",
    "Click here to win a prize!",
    "Exclusive offer! Subscribe now and save 20% on your first purchase. Limited time only!"
]

# Tokenize the sample texts
sample_encodings = tokenizer(sample_texts, truncation=True, padding=True)

# Convert the sample encodings to tensors
sample_input_ids = torch.tensor(sample_encodings['input_ids']).to(device)
sample_attention_mask = torch.tensor(sample_encodings['attention_mask']).to(device)

# Model prediction
model.eval()
with torch.no_grad():
    sample_outputs = model(sample_input_ids, attention_mask=sample_attention_mask)
    sample_logits = sample_outputs.logits
    sample_probs = torch.softmax(sample_logits, dim=1)

# Display the results
for text, prob in zip(sample_texts, sample_probs[:, 1].cpu().numpy()):
    # Convert probability to percentage
    trust_score_percentage = (1 - prob) * 100

    # Set a minimum trust score for non-dark patterns
    min_trust_score = 15 if prob >= 0.5 else 0

    # Ensure that trust score is at least the minimum value
    trust_score_percentage = max(min_trust_score, trust_score_percentage)

    category = "Dark Pattern" if prob >= 0.5 else "Not Dark Pattern"
    print(f"Text: {text}\nPrediction: {category}\nTrust Score: {trust_score_percentage:.2f}%\n")


Text: Limited stock available, buy now!
Prediction: Dark Pattern
Trust Score: 15.00%

Text: Free trial for 30 days, cancel anytime.
Prediction: Dark Pattern
Trust Score: 15.00%

Text: Write a review and get a discount!
Prediction: Not Dark Pattern
Trust Score: 99.71%

Text: Click here to win a prize!
Prediction: Not Dark Pattern
Trust Score: 98.12%

Text: Exclusive offer! Subscribe now and save 20% on your first purchase. Limited time only!
Prediction: Dark Pattern
Trust Score: 15.00%

